In [ ]:
import json
import pandas as pd
import numpy as np
from collections import Counter
import nltk
nltk.download('wordnet')
from nltk.corpus.reader.wordnet import NOUN
from nltk.corpus import wordnet
from nltk.compat import python_2_unicode_compatible
import re
from nltk.stem import WordNetLemmatizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import sklearn.metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import gc
from sklearn.metrics import f1_score, classification_report

#visualization
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns


# A combination of Word lemmatization + LinearSVC model finally pushes the accuracy score past 80%

traindf = pd.read_json("train.JSON")
traindf['ingredients_clean_string'] = [' , '.join(z).strip() for z in traindf['ingredients']]  
traindf['ingredients_string'] = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in traindf['ingredients']]    

In [2]:
corpus = traindf['ingredients_string']
vectorizertr = TfidfVectorizer()
train_vect=vectorizertr.fit_transform(corpus).todense()

In [3]:
target = traindf['cuisine']
train_vect

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [4]:
features = vectorizertr.get_feature_names()

In [5]:
X_train, X_test, y_train, y_test = train_test_split(train_vect, target, test_size=0.30, random_state=42)

In [8]:
from xgboost import XGBClassifier

n_estimators = [500, 1000]
max_depth = [1, 3, 5]
gamma = [0, 0.5]
best_f1 = 0

for est in n_estimators:
    for depth in max_depth:
        for gam in gamma:
            clf = XGBClassifier(
            max_depth=depth,
            min_child_weight=1,
            learning_rate=0.1,
            n_estimators=est,
            silent=True,
            objective='binary:logistic',
            gamma=gam,
            max_delta_step=0,
            subsample=1,
            colsample_bytree=1,
            colsample_bylevel=1,
            reg_alpha=0,
            reg_lambda=0,
            scale_pos_weight=1,
            seed=1,
            )
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            f1 = f1_score(y_test, y_pred, average = 'weighted')
            print(est, depth, gam, f1)
            if f1 > best_f1:
                best_f1 = f1
                best_params = (est, depth, gam)

500 1 0 0.7267005534560586
500 1 0.5 0.7267005534560586
500 3 0 0.7723566728227903
500 3 0.5 0.7736602976560878


KeyboardInterrupt: 